In [1]:
import pandas as pd
train_df = pd.read_csv("titanic/train.csv",encoding="utf-8")
test_df = pd.read_csv("titanic/test.csv",encoding="utf-8")

In [2]:
train_df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [3]:
test_df

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


# 1. 填缺失值

數值: 中位數(最可能的值)
類別: 補上最多的，眾數

# 2. 分特徵

類別: 
    Pclass, Name, Sex, Embarked

數字:
    Age, Sibsp, Parch, Fare

暫時不用:
    Ticket, Cabin

# 3. 處理

多值: 合併、One-Hot-Encoding處理
類別: 有大小關係、只有兩種結果的都可以變成是非題處理

In [4]:
train_df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [5]:
test_df.isna().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [6]:
med = train_df.median()
train_df = train_df.fillna(med)
test_df = test_df.fillna(med)

In [7]:
# 類別：缺失補眾數
most = train_df["Embarked"].value_counts().idxmax()
train_df["Embarked"] = train_df["Embarked"].fillna(most)
test_df["Embarked"] = test_df["Embarked"].fillna(most)

# print(train_df["Embarked"])
# print("="*100)
# print(train_df["Embarked"].value_counts())
# print("="*100)
# print(train_df["Embarked"].value_counts().idxmax())

In [8]:
n = "Braund, Mr. Owen Harris"
n = n.split(",")[-1].split(".")[0]
n.strip()  # strip 可以去掉前後空白換行符號

'Mr'

In [9]:
# 取出名字中的稱謂
def nameflow(n):
    n = n.split(",")[-1].split(".")[0]
    return n.strip()
mid = train_df["Name"].apply(nameflow).value_counts()
reserved = mid[mid > 30].index # ['Mr', 'Miss', 'Mrs', 'Master']
reserved

Index(['Mr', 'Miss', 'Mrs', 'Master'], dtype='object')

In [10]:
# 針對空白處塞 None
def nameflow2(n):
    n = n.split(",")[-1].split(".")[0]
    n = n.strip()
    if n in reserved:
        return n
    else:
        return None
train_df["Name"] = train_df["Name"].apply(nameflow2)
test_df["Name"] = test_df["Name"].apply(nameflow2)

In [11]:
# One-Hot-Encoding處理
x_train_nodrop = pd.get_dummies(train_df,
                                columns=["Name", "Sex", "Embarked"]) # dummy_na=True  可以顯示出NaN欄位
x_test_nodrop = pd.get_dummies(test_df,
                                columns=["Name", "Sex", "Embarked"])

In [12]:
# 處理不想要的欄位
x_train = x_train_nodrop.drop(["PassengerId", "Survived", "Cabin", "Ticket"],
                    axis = 1)
y_train = x_train_nodrop["Survived"]
x_test = x_test_nodrop.drop(["PassengerId", "Cabin", "Ticket"],
                    axis = 1)
testid = x_test_nodrop["PassengerId"]

In [13]:
testid

0       892
1       893
2       894
3       895
4       896
       ... 
413    1305
414    1306
415    1307
416    1308
417    1309
Name: PassengerId, Length: 418, dtype: int64

In [14]:
y_train

0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int64

In [15]:
x_train

,Pclass,Age,SibSp,Parch,Fare,Name_Master,Name_Miss,Name_Mr,Name_Mrs,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,3,22.0,1,0,7.2500,0,0,1,0,0,1,0,0,1
1,1,38.0,1,0,71.2833,0,0,0,1,1,0,1,0,0
2,3,26.0,0,0,7.9250,0,1,0,0,1,0,0,0,1
3,1,35.0,1,0,53.1000,0,0,0,1,1,0,0,0,1
4,3,35.0,0,0,8.0500,0,0,1,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,2,27.0,0,0,13.0000,0,0,0,0,0,1,0,0,1
887,1,19.0,0,0,30.0000,0,1,0,0,1,0,0,0,1
888,3,28.0,1,2,23.4500,0,1,0,0,1,0,0,0,1
889,1,26.0,0,0,30.0000,0,0,1,0,0,1,1,0,0


In [16]:
x_test_nodrop

,PassengerId,Pclass,Age,SibSp,Parch,Ticket,Fare,Cabin,Name_Master,Name_Miss,Name_Mr,Name_Mrs,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,892,3,34.5,0,0,330911,7.8292,NaN,0,0,1,0,0,1,0,1,0
1,893,3,47.0,1,0,363272,7.0000,NaN,0,0,0,1,1,0,0,0,1
2,894,2,62.0,0,0,240276,9.6875,NaN,0,0,1,0,0,1,0,1,0
3,895,3,27.0,0,0,315154,8.6625,NaN,0,0,1,0,0,1,0,0,1
4,896,3,22.0,1,1,3101298,12.2875,NaN,0,0,0,1,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,28.0,0,0,A.5. 3236,8.0500,NaN,0,0,1,0,0,1,0,0,1
414,1306,1,39.0,0,0,PC 17758,108.9000,C105,0,0,0,0,1,0,1,0,0
415,1307,3,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,0,0,1,0,0,1,0,0,1
416,1308,3,28.0,0,0,359309,8.0500,NaN,0,0,1,0,0,1,0,0,1


# Ensemble(組合)
## 1. Boosting(連續)
    AddBoosting: 最後訓練出分類器
    XGBoosting: 
## 2. Bagging(平行)
    Random forest

In [17]:
# 調整參數，找出感覺不錯的解
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
params={
    "n_estimators":range(55, 70, 1),
    "max_depth":range(6, 11)    
}
clf = RandomForestClassifier()
cv = GridSearchCV(clf, params, cv=10, n_jobs=4)
cv.fit(x_train, y_train)
print(cv.best_params_)
print(cv.best_score_)

{'max_depth': 8, 'n_estimators': 66}
0.8395505617977527


In [18]:
# 確認後代入參數
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
clf = RandomForestClassifier(n_estimators=55, max_depth=9)
score = cross_val_score(clf, x_train, y_train, cv=10, n_jobs=4)
print(score)
print(np.average(score))

[0.78888889 0.83146067 0.73033708 0.87640449 0.8988764  0.85393258
 0.82022472 0.78651685 0.87640449 0.83146067]
0.829450686641698


In [19]:
# 帶入答案
clf = RandomForestClassifier(n_estimators=26, max_depth=8)
clf.fit(x_train, y_train)
pre = clf.predict(x_test)
result_df = pd.DataFrame({
    "PassengerId":testid,
    "Survived":pre
})
result_df.to_csv("rf.csv", index=False, encoding="utf-8")
result_df

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [20]:
pd.DataFrame({
    "column":x_train.columns,
    "importance":clf.feature_importances_
})

,column,importance
0,Pclass,0.118403
1,Age,0.113512
2,SibSp,0.064950
3,Parch,0.034683
4,Fare,0.151899
5,Name_Master,0.021588
6,Name_Miss,0.028089
7,Name_Mr,0.170100
8,Name_Mrs,0.030894
9,Sex_female,0.114505


#### 方法看資料而定
knn 適合資料少狀況，調參數方便

Random forest 適合資料多狀況，參數上限較高，強調"特徵重要性"

In [28]:
# 與其調整演算法(效果可能差不多)，不如對資料下手
# 在這裡 MinMaxScaler 效果比 RobustScaler 好

from sklearn.preprocessing import MinMaxScaler
# from sklearn.preprocessing import RobustScaler

# scaler = RobustScaler()
scaler = MinMaxScaler()
x_train_scale = scaler.fit_transform(x_train)
x_test_scale = scaler.transform(x_test)

In [22]:
x_train_scale

array([[1.        , 0.27117366, 0.125     , ..., 0.        , 0.        ,
        1.        ],
       [0.        , 0.4722292 , 0.125     , ..., 1.        , 0.        ,
        0.        ],
       [1.        , 0.32143755, 0.        , ..., 0.        , 0.        ,
        1.        ],
       ...,
       [1.        , 0.34656949, 0.125     , ..., 0.        , 0.        ,
        1.        ],
       [0.        , 0.32143755, 0.        , ..., 1.        , 0.        ,
        0.        ],
       [1.        , 0.39683338, 0.        , ..., 0.        , 1.        ,
        0.        ]])

In [29]:
from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier()
params = {
    "n_neighbors":range(5, 100)
}
cv = GridSearchCV(clf, params, cv=10, n_jobs=4)
cv.fit(x_train_scale,y_train)
print(cv.best_params_)
print(cv.best_score_)

{'n_neighbors': 22}
0.8193508114856428


In [30]:
pre = cv.best_estimator_.predict(x_test_scale)
result_df = pd.DataFrame({
    "PassengerId":testid,
    "Survived":pre
})
result_df.to_csv("knn.csv", index=False, encoding="utf-8")
result_df

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0
